<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training5/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install package
!pip install opencc

In [2]:
from google.colab import drive
drive.mount("gdrive")

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [3]:
!unzip /content/gdrive/MyDrive/translation2019zh.zip

Archive:  /content/gdrive/MyDrive/translation2019zh.zip
replace translation2019zh_train.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace translation2019zh_valid.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# read json

In [16]:
import json
import opencc

def readData(jsonPath):
  datas = []
  chineseCc = opencc.OpenCC('s2hk.json')
  with open(jsonPath,newline="") as js:
    for i in js:
      data = json.loads(i)
      if data["english"].isascii() == False:
        continue
      if len(data["english"].strip()) > 25:
        continue
      if len(data["chinese"].strip()) > 8:
        continue
      data["english"] = data["english"].replace("\\","").strip()
      data["chinese"] = chineseCc.convert(data["chinese"]).strip()
      datas.append(data)
    js.close()
  return datas

In [17]:
def exportDataWithFormat(dataList):
  txt_list = []
  for data in dataList:
    strs = "%s\t%s"%(data["english"],data["chinese"])
    txt_list.append("%s\t%s"%(data["english"],data["chinese"]))
  return txt_list

def writeTotxt(txtList,filePath):
  #wirte file to txt
  with open(filePath,"w",encoding="utf-8") as f:
    for data in txt_list:
      f.write(data)
      f.write("\n")
    f.close()

In [6]:
def readFormatTextToList(filePath):
  input_texts = []
  target_texts = []
  input_chars = set()
  target_chars = set()
  with open(filePath,"r",encoding="utf-8") as f:
    lines = f.read().split("\n") #each line is split by \n
    for line in lines[:min(NUM_SAMPLES,len(lines)-1)]: #num_spaces or line is larger,else will out of range
      input_text, target_text = line.split("\t") #left side of \t is end ,right side is chinese
      
      #each chinese texts need to add \t at the begining and \n at the end of the text
      target_text = "\t" + target_text +"\n"

      #add to input and target list
      input_texts.append(input_text)
      target_texts.append(target_text)

      #loopping the text to get all chars and make one-hot encoding later
      for char in input_text: #for english
        if char not in input_chars:
          input_chars.add(char)

      for char in target_text: # for chinese
        if char not in target_chars:
          target_chars.add(char)
    f.close
    return input_texts,target_texts,input_chars,target_chars

In [18]:
train_data = readData("translation2019zh_train.json")
txt_list = exportDataWithFormat(train_data)
writeTotxt(txt_list,"engChinese.txt")

# Creating the model :Seq2Seq

In [22]:
from keras.models import Model,load_model
from keras.layers import Input, LSTM,Dense
import numpy as np

BATCH_SIZE = 256
EPOCHS = 100
LATENT_DIM = 256
NUM_SAMPLES = 10000
#traning setting
file_path = "engChinese.txt"
#Vetorize data
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

input_texts,target_texts,input_chars,target_chars = readFormatTextToList(file_path)

#handling the data to generate some required info
# sorting the chars 
input_chars = sorted(list(input_chars)) # according to Unicode
target_chars = sorted(list(target_chars)) # according to Unicode

#total chars of input and target
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

#the maxinums sentance of input and target
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Creating Encoder-LSTM
#make each chars has an index
input_token_index = dict([(char , i) for i ,char in enumerate(input_chars)]) # english token with index
target_token_index = dict([(char , i) for i ,char in enumerate(target_chars)]) # chinese token with index
#reverse char-index
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

#Need 3 array with 3-dimension(num_simple,max_seq_text,total_token)
#Encoder-input,Decoder-input,Decoder-output
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_output_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

#zip eng sentance and chinese sentance(eng[0]Chinese[0],eng[1]Chinese[1]...etc)
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  #print(i,(input_text,target_text))
  for t,char in enumerate(input_text):
    #change the array of input data =>i:x,t:y,token:z to 1 = only set z-index to 1
    encoder_input_data[i,t,input_token_index[char]] = 1
  for t,char in enumerate(target_text):
    #chagne the array of target data
    #decoder_target_data is ahead decoder_input_data by one timestemp
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      #only affect t > 0 character,not include \t 
      decoder_output_data[i,t-1,target_token_index[char]] = 1

In [9]:
def trainModel():
  #get model and predict model
  model,encoder_model,decoder_model = createSeq2SeqModel()

  #training the model and save
  #Run traning and complie the model
  model.compile(optimizer="rmsprop",loss="categorical_crossentropy")
  #fit data to model and training
  model.fit([encoder_input_data,decoder_input_data],decoder_output_data,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2)
  #save our model
  model.save("seq2seq.h5")
  encoder_model.save("encoder.h5")
  decoder_model.save("decoder.h5")

In [10]:
def predictTxt(testDataList,correctResult):
  #load the encoder model and decoder model
  encoder_model = load_model("encoder.h5",compile=False)
  decoder_model = load_model("decoder.h5",compile=False)
  index = 0
  for i in testDataList:
    # input_seq=np.zeros((1,max_encoder_seq_length,num_encoder_tokens)) 
    current_seq_data = np.zeros((1,max_encoder_seq_length,num_encoder_tokens)) #1 sentence * max_seq_length(trainData) *num_encoder_tokens(trainData)
    for t,char in enumerate(i):
       current_seq_data[0,t,input_token_index[char]] = 1.0 #set current sentence character to 1 in array
    
    #predict the sentence
    decoded_sentence = decode_sequence(current_seq_data,encoder_model,decoder_model)
    print("English Sentence:\n %s"%(i))
    print("Chinese Sentence(predict):",(decoded_sentence))
    print("Chinese Sentence(Correct):",(correctResult[index]))
    print("-------------------------------------------------")
    index+=1

In [14]:
def createSeq2SeqModel():
  #Define input tensor
  encoder_inputs = Input(shape=(None,num_encoder_tokens)) #Shape 1*num_encoder_tokens
  #encoder LSTM with units = 256 and return the last state
  encoder_lstm = LSTM((LATENT_DIM),return_state=True)
  #output of encoder lstm tensor,return state history and state current
  encoder_outputs,state_h,state_c = encoder_lstm(encoder_inputs)
  #keep states only
  encoder_states = [state_h,state_c]

  #need encoder states as decoder input
  #define decoder lstm and tensor
  decoder_inputs = Input(shape=(None,num_decoder_tokens)) #shape 1*num_decoder_tokens
  #decoder_lstm ,set return everything
  decoder_lstm = LSTM((LATENT_DIM),return_state=True,return_sequences=True)
  #igore the internal state
  decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
  #creating the Dense for full connected layer
  decoder_dense = Dense(num_decoder_tokens,activation="softmax") #num_decoder_tokens * 1
  decoder_outputs = decoder_dense(decoder_outputs) #passing lstm output to decoder dense

  #Define the model
  #input encoder input data and decoder input data and return decoder output data
  model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

  #predict model
  encoder_model = Model(encoder_inputs, encoder_states)
  decoder_state_input_h = Input(shape=(LATENT_DIM,))
  decoder_state_input_c = Input(shape=(LATENT_DIM,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  # lstm
  decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h, state_c]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

  # plot_model(model=model,show_shapes=True)
  # plot_model(model=encoder_model,show_shapes=True)
  # plot_model(model=decoder_model,show_shapes=True)
  return model,encoder_model,decoder_model


In [12]:
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# Training Model(Overfiting~~)

# Dataset is not enough(memeroy not enough too)



In [23]:
if __name__ == "__main__":
  #train the model
  trainModel()

  #load test Data
  testList = readData("translation2019zh_valid.json")
  txt_list = exportDataWithFormat(testList)
  writeTotxt(txt_list,"engChineseTest.txt")
  test_input,test_target,_,_ =readFormatTextToList("engChineseTest.txt")

  #predict
  predictTxt(test_input[:100],test_target[:100])

Epoch 1/100
32/32 [==============================] - 8s 141ms/step - loss: 4.5434 - val_loss: 4.2468
Epoch 2/100
32/32 [==============================] - 3s 105ms/step - loss: 4.2311 - val_loss: 4.2203
Epoch 3/100
32/32 [==============================] - 3s 106ms/step - loss: 4.2030 - val_loss: 4.2110
Epoch 4/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1872 - val_loss: 4.2143
Epoch 5/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1744 - val_loss: 4.2141
Epoch 6/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1636 - val_loss: 4.2038
Epoch 7/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1499 - val_loss: 4.2202
Epoch 8/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1361 - val_loss: 4.1954
Epoch 9/100
32/32 [==============================] - 3s 106ms/step - loss: 4.1219 - val_loss: 4.2103
Epoch 10/100
32/32 [==============================] - 3s 105ms/step - loss: 4.1079 - val_lo